<a href="https://colab.research.google.com/github/Erickrus/llm/blob/main/flowise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=0B0oIs8NS9k

##chatglm 6b

In [ ]:
#@title clone ChatGLM2-6B repository
!git clone https://github.com/THUDM/ChatGLM2-6B
%cd /content/ChatGLM2-6B
!pip3 install -r requirements.txt

In [ ]:
#@title clone pretrained model (chatglm-6b-int4) from huggingface
!git clone https://huggingface.co/THUDM/chatglm2-6b-int4
!cd chatglm2-6b-int4 && rm pytorch_model.bin
#download it separately due to the git clone problem
!cd chatglm2-6b-int4 && wget https://huggingface.co/THUDM/chatglm2-6b-int4/resolve/main/pytorch_model.bin

In [ ]:
#@title define FabUtil

#@markdown FabUtil
import os
import json

import tarfile
from zipfile import ZipFile
from google.colab._system_commands import _shell_line_magic as shell_line_magic


class FabUtil:
  def cust_code(self, codeFilename, content):
    self._ensure_dir(codeFilename)
    with open(codeFilename, 'w') as f:
      f.write(content)

  def fabricate(self, fabs):
    # accept both filename and fabs object
    if type(fabs) == str:
      with open(fabs, "r") as f:
        fabs = json.loads(fabs)
    elif type(fabs) == dict:
      pass

    for i in range(len(fabs["fabs"])):
      fab = fabs["fabs"][i]
      if "cmd" in fab:
        print("%s" % fab["cmd"])
        shell_line_magic("%s" % fab["cmd"])
        #os.system("%s" % fab["cmd"])
        continue

      if "patches" in fab:
        self._patch(fab["srcFilename"], fab["patches"])
        continue

      entryFilename = ""
      srcFilename = fab["srcFilename"]
      if srcFilename.find("::") > 0:
        srcFilename, entryFilename = srcFilename.split("::")
      tgtFilename = fab["tgtFilename"]
      srcFilename = os.path.join(fabs["baseDir"], srcFilename)

      if entryFilename != "":
        self._process_zip_file(srcFilename, entryFilename, tgtFilename)
      else:
        self._ensure_dir(tgtFilename)
        os.system("cp %s %s" % (srcFilename, tgtFilename))
        print("fabricated %s ==> %s" % (srcFilename, tgtFilename))

  def _patch(self, filename, patches):
    changed = False
    with open(filename, 'r') as f:
      lines = f.read().split('\n')
    for patchItem in patches:
      lineNum = patchItem['lineNum']
      fromText = patchItem['fromText']
      toText = patchItem['toText']
      if lines[lineNum-1] == fromText:
        lines[lineNum-1] = toText
        changed = True
    if changed:
      with open(filename, 'w') as f:
        f.write('\n'.join(lines))

  def _ensure_dir(self, tgtFilename):
    dirName = os.path.dirname(tgtFilename)
    if not os.path.exists(dirName):
      os.system("mkdir -p %s " % dirName)

  def _process_zip_file(self, srcFilename, entryFilename, tgtFilename):
    try:
      if srcFilename.lower().find(".tar") > 0 or srcFilename.lower().find(".tgz") > 0:
        fileOp = 'r'
        if srcFilename.lower().endswith('.tar.gz') or srcFilename.lower().endswith('.tgz'): # gzip
            fileOp = 'r:gz'
        elif srcFilename.lower().endswith('.tar.bz2'): # bzip2
            fileOp = 'r:bz2'
        elif srcFilename.lower().endswith('.tar.xz'): # lzma
            fileOp = 'r:xz'
        with tarfile.open(srcFilename, fileOp) as tar:
          self._ensure_dir(tgtFilename)
          with open(tgtFilename, "wb") as f:
            f.write(tar.extractfile(entryFilename).read())
        print("fabricated %s::%s ==> %s" % (srcFilename, entryFilename, tgtFilename))
        return
      if srcFilename.lower().find(".zip") >0:
        with ZipFile(srcFilename, 'r') as z:
          self._ensure_dir(tgtFilename)
          with open(tgtFilename, "wb") as f:
            f.write(z.read(entryFilename))
        print("fabricated %s::%s ==> %s" % (srcFilename, entryFilename, tgtFilename))
        return
    except:
      print("failed %s::%s ==> %s" % (srcFilename, entryFilename, tgtFilename))
      return
    print("not found %s::%s ==> %s" % (srcFilename, entryFilename, tgtFilename))

fb = FabUtil()


In [ ]:
#@title modify web_demo.py
fb.fabricate({
  "baseDir": "/content/",
  "fabs": [
    {
        #@markdown modify tokenizer to load from ./chatglm-6b-int4
        "srcFilename": "web_demo.py",
        "patches": [{
        "lineNum": 6,
        "fromText": 'tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)',
        "toText":   'tokenizer = AutoTokenizer.from_pretrained("./chatglm2-6b-int4", trust_remote_code=True)',
        }]
    },
    {
        #@markdown modify model to load from ./chatglm-6b-int4
        "srcFilename": "web_demo.py",
        "patches": [{
        "lineNum": 7,
        "fromText": 'model = AutoModel.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True).cuda()',
        "toText":   'model = AutoModel.from_pretrained("./chatglm2-6b-int4", trust_remote_code=True).half().cuda()',
        }]
    },
    {
        #@markdown To create a public link, set `share=True` in `launch()`.
        "srcFilename": "web_demo.py",
        "patches": [{
        "lineNum": 108,
        "fromText": 'demo.queue().launch(share=False, inbrowser=True)',
        "toText":   'demo.queue().launch(share=True, inbrowser=True)',
        }]
    },
    {
        #@markdown print(input)
        "srcFilename": "web_demo.py",
        "patches": [{
        "lineNum": 63,
        "fromText": 'def predict(input, chatbot, max_length, top_p, temperature, history, past_key_values):',
        "toText":   'def predict(input, chatbot, max_length, top_p, temperature, history, past_key_values):\n    print(input)',
        }]
    }
  ]
})

In [ ]:
#@title start web_demo
!python3 web_demo.py

/content/ChatGLM2-6B/web_demo.py:91: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=10).style(
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://697006c05b4ac8e9ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
hi
hi
2023-10-17 03:57:14.945714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


##cpolar

In [ ]:
#@title install cpolar

!wget https://static.cpolar.com/downloads/releases/3.3.18/cpolar-stable-linux-amd64.zip?_gl=1*37bbhq*_ga*MTcyNDU3NzMyNC4xNjk3NTEwNjQw*_ga_WF16DPKZZ1*MTY5NzUxMDYzOS4xLjEuMTY5NzUxMDY2MS4zOC4wLjA.
!mv /content/cpolar* /content/cpolar-stable-linux-amd64.zip
!unzip /content/cpolar-stable-linux-amd64.zip
!./cpolar authtoken ......


--2024-05-07 04:20:50--  https://static.cpolar.com/downloads/releases/3.3.18/cpolar-stable-linux-amd64.zip?_gl=1*37bbhq*_ga*MTcyNDU3NzMyNC4xNjk3NTEwNjQw*_ga_WF16DPKZZ1*MTY5NzUxMDYzOS4xLjEuMTY5NzUxMDY2MS4zOC4wLjA.
Resolving static.cpolar.com (static.cpolar.com)... 47.246.24.237, 2404:2280:1b2:0:3::3fe
Connecting to static.cpolar.com (static.cpolar.com)|47.246.24.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423434 (7.1M) [application/zip]
Saving to: ‘cpolar-stable-linux-amd64.zip?_gl=1*37bbhq*_ga*MTcyNDU3NzMyNC4xNjk3NTEwNjQw*_ga_WF16DPKZZ1*MTY5NzUxMDYzOS4xLjEuMTY5NzUxMDY2MS4zOC4wLjA.’

cpolar-stable-linux 100%[===================>]   7.08M   248KB/s    in 31s     

2024-05-07 04:21:20 (236 KB/s) - ‘cpolar-stable-linux-amd64.zip?_gl=1*37bbhq*_ga*MTcyNDU3NzMyNC4xNjk3NTEwNjQw*_ga_WF16DPKZZ1*MTY5NzUxMDYzOS4xLjEuMTY5NzUxMDY2MS4zOC4wLjA.’ saved [7423434/7423434]

Archive:  /content/cpolar-stable-linux-amd64.zip
  inflating: cpolar                  
Authtoken s

In [ ]:
#@title start cpolar

!nohup ./cpolar http 3000 &
#!while true;do clear; ps -ef | grep cpolar ; sleep 5; done

!sleep 5 && ps -ef | grep cpolar

nohup: appending output to 'nohup.out'
root        3829       1  0 04:21 ?        00:00:00 cpolar: master procSHELL=/bin/bash
root        3837     194  0 04:21 ?        00:00:00 /bin/bash -c sleep 5 && ps -ef | grep cpolar
root        3860    3837  0 04:21 ?        00:00:00 grep cpolar


## flowise

In [ ]:
#@title install flowise
!curl -sL https://deb.nodesource.com/setup_20.x | sudo -E bash -
!apt-get install -y nodejs
!npm install -g flowise


added 2661 packages in 6m

293 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New minor version of npm available! 10.5.0 -> 10.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.7.0
npm notice Run npm install -g npm@10.7.0 to update!
npm notice 


In [ ]:
#@title start flowise
!nohup npx flowise start &

nohup: appending output to 'nohup.out'


nohup: appending output to 'nohup.out'


In [ ]:
#!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve &
#!ollama run phi3

In [ ]:
! ollama serve &

time=2024-05-07T04:38:20.032Z level=INFO source=images.go:828 msg="total blobs: 5"
time=2024-05-07T04:38:20.039Z level=INFO source=images.go:835 msg="total unused blobs removed: 0"
time=2024-05-07T04:38:20.039Z level=INFO source=routes.go:1071 msg="Listening on 127.0.0.1:11434 (version 0.1.33)"
time=2024-05-07T04:38:20.040Z level=INFO source=payload.go:30 msg="extracting embedded files" dir=/tmp/ollama433625851/runners
time=2024-05-07T04:38:33.393Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_avx cpu_avx2 cuda_v11 rocm_v60002]"
time=2024-05-07T04:38:33.393Z level=INFO source=gpu.go:96 msg="Detecting GPUs"
time=2024-05-07T04:38:33.406Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-05-07T04:38:33.407Z level=INFO source=gpu.go:96 msg="Detecting GPUs"
time=2024-05-07T04:38:33.415Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-05-07T04:38:33.807Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-05-07T04:38:33.813Z lev